In [43]:
import pandas as pd
import numpy as np
import itertools
from scipy.stats import chi2_contingency
from conditional import csv_format_discrete
from scipy.stats import chi2

In [44]:
df = csv_format_discrete(r"C:\Users\User\Documents\GitHub\ML_FYP\dataset\test_2.dat")

In [45]:
df

,A,B,C,D,E
1,3,3,2,3,1
2,0,2,3,2,2
3,0,2,3,4,3
4,1,2,2,3,0
5,3,4,3,4,0
6,1,1,3,4,4
7,2,1,1,3,4
8,2,2,2,0,2


In [35]:
#takes in a pandas dataframe, collects each column of the dataframe and combine it into a list
def split_(df):
    v = list(df)
    empty_0 = []
    for i in list(df):
        column_of_array = df[i].to_numpy()
        empty_0.append(column_of_array)
    return empty_0

In [36]:
#this function essentially obtains all the combinations of the splitted df
def combinations_(df):
    empty_1 = []
    empty_2 = []
    v = split_(df) #call the function split_
    itertools_combinations = list(itertools.combinations(v, 2))
    return itertools_combinations
#returns a list of combinations

In [37]:
#performs a chi2 test on every combinations in the list, then prints the output
def chi2bool(chi):
    chi = chi2_contingency(chi)
    chi_squared = chi[0]
    dof = chi[2]
    critical_value = chi2.ppf(0.95, dof)
    if chi[0] >  critical_value:
        return("since chi2 =", chi[0] , "is more than the critical value =" , critical_value , ", we reject the null hypothesis, dof=", dof) 
    else: 
        return("since chi2 =" , chi[0] , "is less than the critical value =" , critical_value , ", we do not reject the null hypothesis, dof=", dof)

In [38]:
def tolist(df):
    df = combinations_(df)
    df = np.array(df)
    df = df.tolist()
    return df

: 

In [40]:
for i in p:
    print(chi2bool(i))

ValueError: The internally computed table of expected frequencies has a zero element at (0, 16).